![xarray_logo](https://docs.xarray.dev/en/stable/_static/dataset-diagram-logo.png)

# Introduction II

&copy; Part of **_DKRZ Python Course for Geoscientists_**, licensed by DKRZ under **CC BY-NC-ND 4.0**

Xarray home page: https://xarray.pydata.org/en/stable/index.html <br>
Xarray documentation: https://docs.xarray.dev/en/stable/index.html


Importing modules

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import os, datetime

<br>

----

Define the first spatial Xarray Dataset ds with random data.  
For reproducibility, set the seed to 100000.

In [ ]:
np.random.seed(100000)

time = pd.date_range(start='2023-01-01', periods=2)
lat = [45.,50.,55.,60.]
lon = [0.,5.,10.,15.,20.]

temp = xr.DataArray(np.random.uniform(250,300,40).reshape((2,4,5)),
                   coords={'time': time,
                           'lat': (['lat'], lat),
                           'lon': (['lon'], lon),
                           },
                   name='temp',
                   attrs={'units': 'K', 'standard_name':'surface_temperature'})

prec = xr.DataArray(np.random.uniform(0.001,0.015,40).reshape((2,4,5)),
                   coords={'time': time,
                           'lat': (['lat'], lat),
                           'lon': (['lon'], lon),
                           },
                   name='prec',
                   attrs={'units': 'kg m-2', 'standard_name':'precipitation_amount'})

ds = xr.merge([temp,prec])

ds

----

## Xarray functions/methods


### `transpose` - reordering dimensions

You can reorder the dimensions of a DataArray or Dataset by name with the `transpose()` function.

**Note:**<br>
The order of the dimensions list is not changed but the dimension order of each data variable has changed.

In [ ]:
ds.transpose('lon','lat','time')

In [ ]:
ds.temp.transpose('lat','lon','time')

<br>

### `where` - mask data

Similar to the NumPy `where` function, Xarray provides a `where` function that uses a condition to filter the data.  
You can filter the data by data value range or by a condition related to a dimension for instance.


In [ ]:
var = ds.temp.isel(time=0)
var.data

In [ ]:
var.where((var > 273.15) & (var < 300.)).data

In [ ]:
var.where((var.lon > 10.)).data

<br>

### `isnull` - check where missing values exist

It returns a mask of True/False elements. 

Our _var_ variable does not contain missing values. We now define those values as missing if they are below 273.15.


In [ ]:
var = var.where(var < 273.15)

In [ ]:
print(var.isnull().data)

### `count` - count missing values

Count the data that are not missing values.

In [ ]:
print(var.count().data)

### `notnull` - check where missing values not exist

To check where valid  ("non-missing") values are present, use `notnull`.


In [ ]:
print(var.notnull().data)

In [ ]:
var.data

<br>

### `fillna` - change missing value to a constant number

In [ ]:
var.fillna(-999).data

<br>

### `min(), max(), mean(), sum(), std(), corr(), ...`

Xarray provides a lot of computational functions.


In [ ]:
var = ds.temp
#var

In [ ]:
print(f'min = {var.min().values:6.2f},  max = {var.max().values:6.2f}')

In [ ]:
print(f'std = {var.std().values:6.2f}')

### `groupby, groupby_bins` - group data by dimension

You can group the data by frequency or dimension into bins to do other further computations.

See: https://docs.xarray.dev/en/stable/user-guide/groupby.html

For a better demonstration we read the dataset from the file _rectilinear_grid_2D.nc_ that contains 6-hourly data.

In [ ]:
ds2 = xr.open_dataset('../data/rectilinear_grid_2D.nc')
ds2

In [ ]:
tsurf = ds2.tsurf

It is possible to group the data day by day which allows us to compute the daily means. What does this mean?  
It means that all 6-hourly data of a day is packed together into a group.

First, let's see what happens when we use the groupby method with 'time.day' to resample the time dimension to daily.

In [ ]:
tsurf.groupby('time.day')

<br>The data is grouped to **10 days** 

    40 time steps 6-hourly == 10 days of 6-hour steps

In the next step we use this grouping to compute the daily means of tsurf. Notice that the time dimension name changes to day.

In [ ]:
tsurf_daily_mean = tsurf.groupby('time.day').mean()
tsurf_daily_mean

By the way, we can create a plot of an Xarray Dataset or DataArray with the `.plot()` function. Let's see how the data of the first time step of tsurf looks like.

In [ ]:
tsurf_daily_mean.isel(day=0).plot()

**Note:** <br>
The datetime time accessor is used like

    ds.time.dt.dayofyear
 
### `time.dt.dayofyear`, `time.dt.day`, `time.dt.month`, `time.dt.year`, `...`
In Xarray available datetime shortcuts
- time.dt.dayofyear
- time.dt.day
- time.dt.month
- time.dt.year
- time.dt.season
- ...


In [ ]:
ds.time.dt.dayofyear

In [ ]:
ds2.time.dt.dayofyear

Compute the means of a variable grouped by dayofyear of a multi-year Dataset/DataArray.

In [ ]:
#tsurf.groupby('time.dayofyear').mean()

<br>

<b><font size="+3" color="#ff0000">Exercise: </font></b> 

Use the precip data from the rectilinear_grid_2D.nc file:

1. compute the mean of the variable precip over 'time' and plot it
1. plot only the precip data > 0.0001
1. how many non-missing values exist (exercise # 2.)
1. compute the mean of the variable precip over ('lat','lon') and plot it


In [ ]:
# 1.


In [ ]:
# 2.


In [ ]:
# 3.


<br>

#### Solution:


In [ ]:
# 1. 
cmap = 'Blues'

precip = ds2.precip

precip_mean = precip.mean('time')
precip_mean.plot(cmap=cmap)

In [ ]:
# 2.

precip_mean.where(precip_mean > 0.0001).plot(cmap=cmap)

In [ ]:
# 3.

print(precip_mean.count().data)

In [ ]:
# 4.

precip_fldmean = precip.mean(('lat','lon'))
precip_fldmean.plot()

<br>

----
----

## Applications

<br>

### Daily mean

To compute the **daily mean** of the variable **_tsurf_**:
- group data day-wise
- compute the mean for each day


In [ ]:
day_mean = tsurf.groupby('time.day').mean('time')
day_mean

----

### Global mean

To compute the global (spatial) mean for each time step (like CDO's fldmean)
- compute the mean along the coordinates lat and lon


In [ ]:
day_spatialmean = tsurf.mean(dim=['lon','lat'])
day_spatialmean

In [ ]:
day_spatialmean.plot()

<br>

----

### Weighted global mean

The global mean computed above does not take account to the different area sizes of the lon-lat grid cells. Applications like CDO compute the weighted mean under the hood, but here we have to do it ourselves.

- compute the global weights
- compute the weighted global (field mean) mean
- for correctness add the attribute name weights 
- plot global means with and without weights

Compute the weights

In [ ]:
weights = np.cos(np.deg2rad(tsurf.lat))
weights.name = "weights"
#weights

Compute the weighted data of the variable tsurf with the xarray.DataArray method `weigthed`.

In [ ]:
tsurf_weighted = tsurf.weighted(weights)
tsurf_weighted

Compute the weighted global mean and keep all attributes.

In [ ]:
tsurf_weighted_mean = tsurf_weighted.mean(('lon', 'lat'), keep_attrs=True)
tsurf_weighted_mean

For comparison we plot the global mean and weighted global mean in the same plot frame. Before we can do this we have to import Matplotlibs pyplot.

In [ ]:
plt.figure(figsize=(8,3))
plt.ylim(275.,288.)

tsurf_weighted_mean.plot(label='weighted')
tsurf.mean(('lon', 'lat')).plot(label='unweighted')
#day_spatialmean.plot(label='unweighted')

plt.legend();

This is a good example to demonstrate how important it is to consider the different size of the grid cells o avoid distortion of the data.

----

### Climatology

For our next example we use a historical and scenario dataset from CMIP6 that contain more time steps.

In [ ]:
dir_data  = "/work/kv0653/k204045/data/Tutorial_Data/"
# historical data
fnameh = "hist_em_LR_temp2.nc"
# scenario data
fname  = "ssp245_em_LR_temp_1995-2100.nc"

First, we extract a 20-year time range (1995-2014).

In [ ]:
dsh = xr.open_dataset(dir_data + fnameh)
dsh = dsh.sel(time=slice('1995-01-01','2014-12-31'))

Xarray allows us to group the time steps monthly-wise with the `groupby` method and compute the monthly means. 

In [ ]:
clim_xr = dsh.groupby('time.month').mean()

To account for the different grid cell sizes, we calculate the variable weighted this time.

Compute the weights.

In [ ]:
weights = np.cos(np.deg2rad(dsh.lat))

Compute the weighted data.

In [ ]:
clim_xr_wgt = clim_xr.weighted(weights)

Compute the spatial mean.

In [ ]:
clim_xr_mean = clim_xr_wgt.mean(('lat','lon'))

Plot the result.

In [ ]:
clim_xr_mean.tas.plot()

### Anomaly

Open the scenario file and extract the time range 2015-2100.

In [ ]:
ds = xr.open_dataset(dir_data + fname)
ds = ds.sel(time=slice('2015-01-01','2100-12-31'))

Compute the spatial mean of the weighted data.

In [ ]:
tas_xr_wgt = ds.tas.weighted(weights).mean(('lat','lon'))

Compute the anomaly. This is done by subtracting the monthly climatology from the monthly grouped data.

In [ ]:
anom_xr = tas_xr_wgt.groupby('time.month') - clim_xr_mean

Compute the yearly means.

In [ ]:
anom_xr_ymean = anom_xr.resample(time='Y').mean()

Plot the result.

In [ ]:
anom_xr_ymean.tas.plot()

<br>

----
----

See also:

- Project Pythia Computations and Masks with Xarray https://foundations.projectpythia.org/core/xarray/computation-masking.html
- Tutorials and Videos https://docs.xarray.dev/en/stable/tutorials-and-videos.html
- DKRZ tutorials https://data-infrastructure-services.gitlab-pages.dkrz.de/tutorials-and-use-cases/Tutorials.html
- Pangeo Xarray Tutorial http://gallery.pangeo.io/repos/pangeo-data/pangeo-tutorial-gallery/xarray.html
- Copernicus https://ecmwf-projects.github.io/copernicus-training-c3s/reanalysis-climatology.html#anomaly-calculation
